In [1]:
import os
import numpy as np
import json

# ⭐️ Pulling in processed lyrics JSON from DeepRapper

In `deeprapper_dict` the keys to note are 
- `Song` : maps the lyrical content to beats
    - example: [BEAT] In [BEAT] whirl [BEAT] whirl [BEAT] [SEP] whirl [BEAT] outside [BEAT] [SEP] [BEAT] fan [BEAT] lost [BEAT] [BEAT] when

<br />

- `finals`: maps the phonetic content to beats (not perfect phonetic translation bc of the chinese / english conversion)
    - example: [BEAT] ai [BEAT] an [BEAT] o [BEAT] ong [SEP] an o [BEAT] i [BEAT] ai [SEP] [BEAT] i [BEAT] i [BEAT] ai [BEAT] i

<br />

- `sentence`: maps the given word to the bar it is in
    - example: 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3

<br />

NOTE: we do not know what `relative position` encodes. Example output:
- [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [SEP] 0 1 [BEAT] 2 [BEAT] 3 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] ] 4 5 6 [BEAT] 7 [BEAT] 8 9 [BEAT] 10 [BEAT] 11 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 5 6 [BEAT] 7 8 [BEAT] 9

Do the beats line up to the bar? Probably not, because each [BEAT] number is increasing.

<br /><br />

#### super long raw song string from lyrics.json in deep rapper

In [2]:
one_song = " {'Singer': 'rap_Kung Fu Fat KungFuPen', 'Album': 'Light Up Live Action LIVE Compilation', 'Song Title': 'Be Like Flame (Light Up Live Version))', 'Song': '[BEAT] In [BEAT] whirl [BEAT] whirl [BEAT] [SEP] whirl [BEAT] outside [BEAT] [SEP] [BEAT] fan [BEAT] lost [BEAT] [BEAT] when [BEAT] [BEAT] Empty was born [BEAT] The net of life [BEAT] [BEAT] covered [BEAT] covered [SEP] [BEAT] The [BEAT] of [BEAT] in the dream [BEAT] [BEAT] wild [BEAT] beast is sinking [BEAT] sleeping in [BEAT] remember [BEAT] remember [BEAT] deep [BEAT] place [SEP] [BEAT] where [BEAT] says [BEAT] reason [BEAT] think [BEAT] home [BEAT] garden only [BEAT] because of [BEAT] ] For [BEAT] you [BEAT] far [BEAT] way [BEAT] and [BEAT] come [SEP] I [BEAT] we [BEAT] do not [BEAT] need [BEAT] to understand [SEP] [BEAT] in [BEAT] It hurts [BEAT] It hurts [SEP] [BEAT] It hurts [BEAT] It hurts [BEAT] outside [SEP] You saw [BEAT] from [BEAT] smoke [BEAT] to [SEP] [BEAT] ] No [BEAT] method [BEAT] forbearance [BEAT] subject to [BEAT] [BEAT] not only [BEAT] is [BEAT] one [BEAT] step [BEAT] step [BEAT] back [BEAT] let [SEP][ BEAT] is [BEAT] with [BEAT] love [BEAT] with [BEAT] name [BEAT] meaning [BEAT] sacrifice [BEAT] sacrifice [BEAT] with [BEAT] love [BEAT] with [BEAT] name [BEAT] Righteousness [BEAT] left [BEAT] forget [SEP] heart [BEAT] there is a long bank thousand [BEAT] [BEAT] collapse [BEAT] in the ant [BEAT] hole [SEP] ash [BEAT] ash [BEAT] yes [BEAT] You [BEAT] are [BEAT] I am [BEAT] Yesterday [BEAT] day [SEP] [BEAT] day [BEAT] hall [BEAT] [BEAT] destroyed [BEAT] in [BEAT] once [BEAT] ] place [BEAT] square [SEP] [BEAT] this [BEAT] bar [BEAT] road [BEAT] from [BEAT] to [BEAT] to walk [BEAT] to move [SEP] straight [BEAT] to [BEAT] raw [BEAT] live out [BEAT] and show some kind of [BEAT] like [BEAT] sign [SEP] strangers [BEAT] born [BEAT] people you [BEAT] when you come to [BEAT] bring [BEAT] letter [BEAT] things [SEP] ]', 'Song reverse': 'In the [BEAT] vortex [BEAT] whirl [BEAT] outside the [BEAT] [SEP] [BEAT] whirlpool [SEP] cover [BEAT] cover [BEAT] net [BEAT] life [BEAT] life When being empty [BEAT] [BEAT] at [BEAT] lost [BEAT] fan [BEAT] [SEP] [BEAT] deep [BEAT] memory [BEAT] remember [BEAT] sleeping [BEAT] Shen Zheng beast [BEAT] ] Wild [BEAT] in [BEAT] in [BEAT] dream [BEAT] [SEP] to [BEAT] and [BEAT] way [BEAT] far [BEAT] you [BEAT] for [BEAT] because of [BEAT] Gion [BEAT] My family [BEAT] wants to [BEAT] and [BEAT] means [BEAT] what [BEAT] [SEP] knows [BEAT] needs [BEAT] does not [BEAT] you [BEAT] I [SEP] are in pain[ BEAT] pain [BEAT] outside [BEAT] [SEP] [BEAT] pain [BEAT] pain [BEAT] [SEP] to [BEAT] after fancy smoke [BEAT] nitrate [BEAT] from you [SEP] let [BEAT] step back [BEAT] step [BEAT] step [BEAT] one [BEAT] is [BEAT] only not [BEAT] of [BEAT] subject to [BEAT] endure [BEAT] method [BEAT] no [BEAT] [SEP ] Forget [BEAT] bequeath [BEAT] meaning [BEAT] name [BEAT] [BEAT] love [BEAT] sacrifice [BEAT] sacrifice [BEAT] meaning [BEAT] name [BEAT] [BEAT] love [BEAT] With [BEAT] it is [BEAT] [SEP] cave [BEAT] ants in [BEAT] collapse [BEAT] [BEAT] Qiandichang has [BEAT] heart [SEP] days [BEAT] Yesterday [BEAT] It's me [BEAT] is [BEAT] you [BEAT] is [BEAT] in ash [BEAT] ash [SEP] [BEAT] in the [BEAT] land [BEAT] destroyed [BEAT] in [BEAT] on [ BEAT] Church [BEAT] Days [BEAT] [SEP] Actively feel yourself [BEAT] walk [BEAT] yourself [BEAT] from [BEAT] road [BEAT] strips [BEAT] this [BEAT][SEP] sign [BEAT] something like [BEAT]BEAT] come out [BEAT] give birth [BEAT] to [BEAT] straight [SEP] thing [BEAT] believe [BEAT] when bring [BEAT] come [BEAT] you [BEAT] give birth to [BEAT] stranger [SEP] ', 'finals': '[BEAT] ai [BEAT] an [BEAT] o [BEAT] ong [SEP] an o [BEAT] i [BEAT] ai [SEP] [BEAT] i [BEAT] i [BEAT] ai [BEAT] i [BEAT] ] ong ei en [BEAT] in [BEAT] i ang [BEAT] u [BEAT] ai [SEP] [BEAT] en [BEAT] ong [BEAT] e [BEAT] ie [BEAT] ou en en [BEAT] ei ai [BEAT] i [BEAT] i [BEAT] en [BEAT] u [SEP] [BEAT] o [BEAT] ei [BEAT] i [BEAT] ang [BEAT] a [BEAT] an i [BEAT] in [ BEAT] ei [BEAT] i [BEAT] an [BEAT] ao [BEAT] er [BEAT] ai [SEP] o [BEAT] en [BEAT] u [BEAT] i [BEAT] ao ong [SEP] [BEAT] ai [BEAT] en [BEAT] ong ong [SEP] [BEAT] en [BEAT] ong i [BEAT] ai [SEP] i ong [BEAT] ao [BEAT] an ong an o [BEAT] ai [SEP] [ BEAT] u [BEAT] a [BEAT] en [BEAT] ou [BEAT] e [BEAT] ui [BEAT] i [BEAT] i [BEAT] u [BEAT] u [BEAT] ei [BEAT] ang [SEP] [BEAT] i [BEAT] i [BEAT] ai [BEAT] e [BEAT] in [BEAT] i [BEAT] i [BEAT] en [BEAT] i [BEAT] ai [BEAT] e [BEAT] in [BEAT] ] i [BEAT] i[BEAT] ang [SEP] in [BEAT] ou ang i an [BEAT] i [BEAT] ei [BEAT] ii [BEAT] ie [SEP] ei [BEAT] in ong [BEAT] i [BEAT] i [BEAT] ] i [BEAT] oi [BEAT] o [BEAT] an [SEP] [BEAT] an [BEAT] ang [BEAT] ou [BEAT] ai [BEAT] ei ei [BEAT] o [BEAT] ee [BEAT] ang [SEP] [BEAT] e [BEAT] ao [BEAT] u [BEAT] i [BEAT] i [BEAT] ou [BEAT] ii an ong [SEP] i [BEAT] ao [BEAT] en [BEAT] ou [ BEAT] an ou ong [BEAT] ang [BEAT] en [SEP] o [BEAT] en [BEAT] en i [BEAT] ai [BEAT] i ai [BEAT] e in [BEAT] u [SEP]', ' Reverse finals': 'ong [BEAT] o [BEAT] an [BEAT] ai [BEAT] [SEP] ai [BEAT] i [BEAT] o an [SEP] ai [BEAT] u [BEAT] ang i [BEAT] in [BEAT] en ei ong [BEAT] i [BEAT] ai [BEAT] i [BEAT] i [BEAT] [SEP] u [BEAT] en [BEAT] i [BEAT] i [BEAT] ai ei [BEAT] en en ou [BEAT] ] ie [BEAT] e [BEAT] ong [BEAT] en [BEAT] [SEP] ai [BEAT] er [BEAT] ao [BEAT] an [BEAT] i [BEAT] ei [BEAT] in [BEAT] i an [BEAT] a [BEAT] ang [BEAT] i [BEAT] ei [BEAT] o [BEAT] [SEP] ong ao [BEAT] i [BEAT] u [BEAT] en [BEAT] o [SEP] ong ong [ BEAT] en [BEAT] ai [BEAT] [SEP] ai [BEAT] i ong [BEAT] en [BEAT] [SEP] ai [BEAT] o an ong an [BEAT] ao [BEAT] ong i [SEP] ang [BEAT] ei [BEAT] u [BEAT] u [BEAT] i [BEAT] i [BEAT] iu [BEAT] e [BEAT] ou [BEAT] en [BEAT] a [BEAT] u [BEAT] [SEP] ang [BEAT] i [BEAT] i [BEAT] in [BEAT] e [BEAT] ai [BEAT] i [BEAT] en [BEAT] i [BEAT] i [BEAT] in [BEAT] e [BEAT] ai [ BEAT] i [BEAT]i [BEAT] [SEP] ie [BEAT] ii [BEAT] ei [BEAT] i [BEAT] an i ang ou [BEAT] in [SEP] an [BEAT] o [BEAT] io [BEAT] i [BEAT] i [BEAT] i [BEAT] ong in [BEAT] ei [SEP] ang [BEAT] ee [BEAT] o [BEAT] ei ei [BEAT] ai [BEAT] ou [BEAT] ang [BEAT] an [BEAT] [SEP] ong an ii [BEAT] ou [BEAT] i [BEAT] i [BEAT] u [BEAT] ao [BEAT] e [BEAT] [SEP] en ​​[BEAT] ang [BEAT] ong ou an [BEAT] uo [BEAT] en [BEAT] ao [BEAT] i [SEP] u [BEAT] in e [BEAT] ai i [BEAT] ai [BEAT] i en [BEAT] en [BEAT] o [SEP]', ' sentence': '0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 1 1 1 11 1 1 1 1 1 1 1 11 1 11 11 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 14 14 relative position[BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [SEP] 0 1 [BEAT] 2 [BEAT] 3 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] ] 4 5 6 [BEAT] 7 [BEAT] 8 9 [BEAT] 10 [BEAT] 11 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 5 6 [BEAT] 7 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [BEAT] 12 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 6 [BEAT] 7 [ BEAT] 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [BEAT] 12 [BEAT] 13 [SEP] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 5 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 3 [SEP] [BEAT] 0 [BEAT] 1 2 [BEAT] 3 [SEP] 0 1 [BEAT] 2 [BEAT] 3 4 5 6 [BEAT] 7 [SEP] [ BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 6 [BEAT] 7 [BEAT] 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [BEAT] 12 [SEP ] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 [BEAT] 6 [BEAT] 7 [BEAT] 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [ BEAT] 12 [BEAT] 13 [BEAT] 14 [SEP] 0 [BEAT] 1 2 3 4 [BEAT] 5 [BEAT] 6[BEAT] 7 8 [BEAT] 9 [SEP] 0 [BEAT] 1 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 [BEAT] 6 7 [BEAT] 8 [BEAT] 9 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 5 [BEAT] 6 [BEAT] 7 8 [BEAT] 9 [SEP] [BEAT] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [ BEAT] 4 [BEAT] 5 [BEAT] 6 7 8 9 [SEP] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 4 [BEAT] 5 6 7 [BEAT] 8 [BEAT] 9 [SEP] 0 [ BEAT] 1 [BEAT] 2 3 [BEAT] 4 [BEAT] 5 6 [BEAT] 7 8 [BEAT] 9 [SEP]', 'Reverse relative position': '0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] [SEP] 0 [BEAT] 1 [BEAT] 2 3 [SEP] 0 [BEAT] 1 [BEAT] 2 3 [BEAT] 4 [BEAT] 5 6 7 [BEAT] 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [BEAT] [SEP] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 5 [BEAT] 6 7 8 [BEAT] ] 9 [BEAT] 10 [BEAT] 11 [BEAT] 12 [BEAT] [SEP] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 [BEAT] 6 [BEAT] 7 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [BEAT] 12 [BEAT] 13 [BEAT] [SEP] 0 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 [SEP] 0 1 [ BEAT] 2 [BEAT] 3 [BEAT] [SEP] 0 [BEAT] 1 2 [BEAT] 3 [BEAT] [SEP] 0 [BEAT] 1 2 3 4 [BEAT] 5 [BEAT] 6 7 [SEP] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 [BEAT] 6 7 [BEAT] 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [BEAT] 12 [BEAT] [SEP ] 0 [BEAT] 1 [BEAT] 2 [BEAT] 3 [BEAT] 4 [BEAT] 5 [BEAT] 6 [BEAT] 7 [BEAT] 8 [BEAT] 9 [BEAT] 10 [BEAT] 11 [BEAT] 12 [BEAT] 13 [BEAT] 14 [BEAT] [SEP] 0 [BEAT] 1 2 [BEAT] 3 [BEAT] 4[BEAT] 5 6 7 8 [BEAT] 9 [SEP] 0 [BEAT] 1 [BEAT] 2 3 [BEAT] 4 [BEAT] 5 [BEAT] 6 [BEAT] 7 8 [BEAT] 9 [SEP] 0 [ BEAT] 1 2 [BEAT] 3 [BEAT] 4 5 [BEAT] 6 [BEAT] 7 [BEAT] 8 [BEAT] 9 [BEAT] [SEP] 0 1 2 3 [BEAT] 4 [BEAT] 5 [BEAT] 6 [BEAT] 7 [BEAT] 8 [BEAT] 9 [BEAT] [SEP] 0 [BEAT] 1 [BEAT] 2 3 4 [BEAT] 5 6 [BEAT] 7 [BEAT] 8 [BEAT] 9 [SEP] 0 [BEAT] 1 2 [BEAT] 3 4 [BEAT] 5 [BEAT] 6 7 [BEAT] 8 [BEAT] 9 [SEP]', 'Drumbeat': '1 0 2 0 3 0 4 0 0 0 0 5 0 6 0 0 7 0 8 0 9 0 10 0 11 0 0 0 12 0 13 0 0 14 0 15 0 0 16 0 17 0 18 0 19 0 20 0 0 0 21 0 0 22 0 23 0 24 0 25 0 0 26 0 27 0 28 0 29 0 30 0 31 0 0 32 0 33 0 34 0 35 0 36 0 37 0 38 0 0 0 39 0 40 0 ​​41 0 42 0 0 0 43 0 44 0 45 0 0 0 46 0 47 0 0 48 0 0 0 0 49 0 50 0 0 0 0 51 0 0 52 0 53 0 54 0 55 0 56 0 57 0 0 58 0 59 0 60 0 61 0 62 0 63 0 0 64 0 65 0 66 0 67 0 68 0 69 0 70 0 71 0 72 0 73 0 74 0 75 0 76 0 77 0 78 0 0 0 79 0 0 0 0 80 0 81 0 82 0 0 83 0 0 0 84 0 0 85 0 86 0 87 0 88 0 0 89 0 90 0 0 91 0 92 0 93 0 94 0 95 0 0 96 0 97 0 0 98 0 0 99 0 100 0 101 0 102 0 103 0 104 0 105 0 0 0 0 0 0 106 0 107 0 108 0 0 109 0 0 0 110 0 111 0 0 0 112 0 113 0 0 114 0 115 0 0 116 0 0 117 0 0', 'Drum reverse': '0 1 0 2 0 3 0 4 0 0 5 0 6 0 0 0 0 7 0 8 0 0 9 0 10 0 0 0 11 0 12 0 13 0 14 0 15 0 0 16 0 17 0 18 0 19 0 0 20 0 0 0 21 0 22 0 23 0 24 0 25 0 0 26 0 27 0 28 0 29 0 30 0 31 0 32 0 0 33 0 34 0 35 0 36 0 37 0 38 0 0 0 39 0 40 0 ​​41 0 42 0 0 0 0 43 0 44 0 45 0 0 46 0 0 47 0 48 0 0 49 0 0 0 0 50 0 51 0 0 0 0 52 0 53 0 54 0 55 0 56 0 57 0 0 58 0 59 0 60 0 61 0 62 0 63 0 0 64 0 65 0 66 0 67 0 68 0 69 0 70 0 71 0 72 0 73 0 74 0 75 0 76 0 77 0 78 0 0 79 0 0 80 0 81 0 82 0 0 0 83 0 0 0 84 0 85 0 0 86 0 87 0 88 0 89 0 0 90 0 0 0 91 0 0 92 0 93 0 0 94 0 95 0 96 0 97 0 98 0 0 0 0 0 99 0 100 0 101 0 102 0 103 0 104 0 105 0 0 106 0 107 0 0 0 108 0 0 109 0 110 0 111 0 0 0 112 0 0 113 0 0 114 0 115 0 0 116 0 117 0 0', 'valid': True} "

#### processing `deeprapper_dict`

In [3]:
key_item = []
split = one_song.split(":")
for i in range(1, len(split)):

    if i == 1:
        key = split[i - 1].split("', ")[0]
        item = split[i].split("', ")[0]
    else:
        key = split[i - 1].split("', ")[1]
        item = split[i].split("', ")[0]

        if "relative position" in item:
            concat = key + item
            two = concat.split("relative position")
            sentence = two[0]
            rel = two[1]
            rel = "'relative position' '" + rel

            key_item.append(sentence)
            key_item.append(rel)
            pass
        
    key_item.append(key + item)

deeprapper_dict = {}
for elm in key_item:
    split = elm.split("'")

    if len(split) == 3:
        deeprapper_dict[str(split[1])] =  split[2]
    else:
        deeprapper_dict[str(split[1])] =  split[3]

In [4]:
deeprapper_dict

{'Singer': 'rap_Kung Fu Fat KungFuPen',
 'Album': 'Light Up Live Action LIVE Compilation',
 'Song Title': 'Be Like Flame (Light Up Live Version))',
 'Song': '[BEAT] In [BEAT] whirl [BEAT] whirl [BEAT] [SEP] whirl [BEAT] outside [BEAT] [SEP] [BEAT] fan [BEAT] lost [BEAT] [BEAT] when [BEAT] [BEAT] Empty was born [BEAT] The net of life [BEAT] [BEAT] covered [BEAT] covered [SEP] [BEAT] The [BEAT] of [BEAT] in the dream [BEAT] [BEAT] wild [BEAT] beast is sinking [BEAT] sleeping in [BEAT] remember [BEAT] remember [BEAT] deep [BEAT] place [SEP] [BEAT] where [BEAT] says [BEAT] reason [BEAT] think [BEAT] home [BEAT] garden only [BEAT] because of [BEAT] ] For [BEAT] you [BEAT] far [BEAT] way [BEAT] and [BEAT] come [SEP] I [BEAT] we [BEAT] do not [BEAT] need [BEAT] to understand [SEP] [BEAT] in [BEAT] It hurts [BEAT] It hurts [SEP] [BEAT] It hurts [BEAT] It hurts [BEAT] outside [SEP] You saw [BEAT] from [BEAT] smoke [BEAT] to [SEP] [BEAT] ] No [BEAT] method [BEAT] forbearance [BEAT] subject to [B

# ⭐️ Pulling in lyrics data from MCFlow

The final dictionary for the English cleaned data is `mc_processed_dict`.

Its keys are `['recip', 'stress', 'tone', 'break', 'rhyme', 'ipa', 'lyrics', 'hype', 'sentence', 'song', 'artist']` and follow the same structure as [the MCFlow columns](https://github.com/Computational-Cognitive-Musicology-Lab/MCFlow/blob/master/Humdrum/LaurynHill_DooWopThatThing.rap). For more documentation on the columns, check the [MCFlow README](https://github.com/Computational-Cognitive-Musicology-Lab/MCFlow/blob/master/Humdrum/README.md).

<br />

Some considerations....
- We added the `sentence` column. It follows the same structure as `deeprapper_dict[sentence]`
- In DeepRapper's `deeprapper_dict`, keys map to strings. ➡️ In our `mc_processed_dict`, keys map to lists.

<br />
<br />


In [5]:
directory = './MCFlow-master/Humdrum'
files=[]
songTitles=[]
artists=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        extension=f.split('/')[-1].split('_')
        if len(extension)>1:
            artists.append(extension[0])
            songTitles.append(extension[1].split('.')[0])
            files.append(f)

In [6]:
def pad(line):
    while len(line)!=8:
        line.append('.')
    return line

def encodeSong(songFile):   
    with open(songFile) as f:
        lines = f.readlines()
    col_names = [col[:-1] for col in lines[0].split("**")[1:]]
    col_names.append("pos")
    col_names.append('beat')
    col_names.append('sentence')
    
    # print('Col names length', col_names)

    processed=[]
    count=0
    rel = 0
    beat=0
    for i in range(5,len(lines)):
        line = lines[i].split('\t')

        # add rel line pos
        line.append(rel)
        
        # add beats to lyrics, rhymes, relative position
        if line[1] == str(1):
            line[6] = "[BEAT] " + line[6] # lyrics
            line[5] = "[BEAT] " + line[5] #rhyme
            line[8] = "[BEAT] " + str(line[8]) # rel position

        #end of file pass
        if line[0] == "*-":
            break

        # skip over bar
        if line[0][0]=='=':
            continue

        rel += 1
        # count for sentence 
        if line[5] == 'R':
            rel = 0
            count += 1
            beat+=1
            line.append(beat)
        else:
            line.append('0')

        # add count for sentence
        line.append(count)
        processed.append(line)
        
    processed= np.vstack(processed)
    mc_processed_dict = {}
    for i in range(len(col_names)):
        mc_processed_dict[col_names[i]]=processed[:,i]
    return mc_processed_dict

In [7]:
song = encodeSong(files[20])
song['beat']

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '2', '0', '0',
       '0', '0', '0', '0', '3', '0', '0', '0', '0', '0', '0', '4', '0',
       '0', '0', '0', '0', '0', '5', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '6', '0', '0', '0', '0', '0', '0', '7',
       '0', '0', '0', '0', '0', '0', '8', '0', '0', '0', '0', '0', '0',
       '9', '0', '0', '0', '0', '0', '0', '10', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '11', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '12', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

# ⭐️ Create list of 124 JSON dictionaries of MCFlow dataset
`songList` is the list of songs formatted above.

In [8]:
songList = []
for i, songFile in enumerate(files):
    dictionary = encodeSong(songFile)
    dictionary['title'] = songTitles[i]
    dictionary['artist']=artists[i]
    songList.append(dictionary)

In [9]:
songList[0].keys()

dict_keys(['recip', 'stress', 'tone', 'break', 'rhyme', 'ipa', 'lyrics', 'hype', 'pos', 'beat', 'sentence', 'title', 'artist'])

In [10]:
songList[1]['sentence']

array(['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2',
       '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '4', '4',
       '4', '4', '4', '5', '5', '5', '5', '5', '5', '5', '5', '5', '5',
       '5', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6',
       '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '6', '7',
       '7', '7', '7', '7', '7', '7', '8', '8', '9', '9', '9', '10', '10',
       '10', '11', '11', '11', '11', '11', '11', '11', '12', '12', '12',
       '12', '12', '12', '13', '13', '13', '13', '13', '13', '13', '13',
       '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13',
       '13', '13', '13', '14', '14', '14', '14', '14', '14', '14', '14',
       '14', '14', '14', '14', '14', '14', '15', '15', '15', '15', '16',
       '16', '16', '16', '17', '18', '18', '18', '18', '1

# ⭐️ Formatting to match the DeepRapper lyrics.json format

```
{ 
    ‘title’: __, 
    ‘artist”:__, 
    ‘lyric’: string of all lyrics (full words) should be done by concatenating by ‘-‘ in MC files, 
    ‘lyric-reverse’: same but in reverse order, 
    ‘vowel’: last syllable in the word, 
    ‘vowel-reverse’: same but in reverse order, 
    ‘sentence’: Indicates the sentence/phrase index that each word/beat/pad is a part of, 
    ‘pos’: indicates the position in the sentence for each lyric that’s not a [beat] or [pad], 
    ‘pos-reverse’: same but in reverse, 
    ‘beat’: 0 when no beat and # in increasing order when there is a beat, 
    ‘beat-reverse’: same but in reverse
}
```

In [11]:
processed_list = []

for song in songList:
    
    processed = {}
    processed['title'] = song['title']
    processed['artist'] = song['artist']
    processed['lyric']= ' '.join(song['lyrics'])
    processed['lyric-reverse']= ' '.join(reversed(song['lyrics'].tolist()))
    processed['vowel'] = ' '.join(song['ipa'])
    processed['vowel-reverse'] = ' '.join(reversed(song['ipa'].tolist()))
    processed['sentence'] = ' '.join(song['sentence'])
    processed['pos'] = ' '.join(song['pos'])
    processed['pos-reverse'] = ' '.join(reversed(song['pos'].tolist()))
    processed['beat']= ' '.join(song['beat'])
    processed['beat-reverse']= ' '.join(reversed(song['beat'].tolist()))
    processed['valid']=True
    processed_list.append(processed)

    # format lyric
jsonString = json.dumps(processed_list)
jsonFile = open("mcflowLyrics.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [12]:
jsonFile = open("mcflowLyrics2.json","w")

line = jsonString
line = line[1:-1]
line = line.replace("\"", "'")
line = line.replace("true", "True")
songs = line.replace("}, {","}\n{")
songs = songs.split("\n")
stringfile = json.dumps(songs)
jsonFile.write(stringfile)
jsonFile.close()

In [19]:
lyric_set =set()
for i in range(len(processed_list)):
    a=processed_list[i]['lyric'].split(' ')
    lyric_set.update(a)
for elem in lyric_set:
    print(elem)


act.
_with^
-us
-bre-
_hun-
-fend
blurr-
-rec-
5
sets
!Sacramento
-tels
Coo-
psych
cole
are.
-lly
!2Pacalypse
grasp
-llis
!MC:112
rhyme.
slug
stea-
ye
wake
-sci-
bags.
east-
God-
-ringe
thing".
floor
Pam
roads
sucks
-form
Should
mad,
Twen-
min-
-vance
years}
-per's
com-
Lex-
fame
pub
Sits
-not-
uh
vo-
bricks
plain
nu-
vin-
down(
c_
sys-
hops.
-while
-to-
card
two-
hug
smack
-or's
jour-
Er-
-cra-
rhy-
-dans
homeboy
-fen-
feel
bees
ought
If
ber-
Ha-
-ggles
-keep
Banks
Lunch-
-cess
dial
grow
care-
this.
cup;
-talist
jeans
full
Jaws
-mas-
Ho-
bombs.
*MM140
ear
hood
row
-yo
-pale
Tha-
nien
-arm
-cere
pimps
bridge
Meu-
bet
slow.
-co
Bill-
text
Boi
-firmed
mugs
N'-
was[
-sy's
supposed
tick-
-tan
Told
air.
fix
new)
Lane
four?
rhymes
brain-
Dogg
Ar-
vol-
-gies
-for-
-tra-
-xin'
-beat
-mem-
-cret.
honk-
fish
!MC:Money
Spea-
batch
-give-
12
prints
-er-
ride
reach
ner-
S.
psy-
this+
role,
passed
-mey
sweet-
bass
:I
square
Dia-
stop,
le-
drops.
Johnny
-nnaise
thoughts
squea-
Thurs-
-ful)
Brass
Kea